In [67]:
!pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.5/7.5 MB 23.0 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=39eae291e8b51c5536c7241e3ed6acfd0447ff0c023a4e39060dcc325d372de3
  Stored in directory: c:\users\mdc20\appdata\local\pip\cache\wheels\95\8e\3e\6372bf25c0ed7afad69bc3612e851d17314f5a77655811fd99
Successfully built sklearn


In [3]:
import numpy as np
import cv2
import scipy


In [4]:
FOLDER_PATH = 'C:\\Users\\mdc20\\Downloads\\img'
CLASS_LIST_21 = ["Tee","Tank","Dress","Shorts","Skirt","Hoodie","Jumpsuit","Sweater","Blazer","Striped","Cardigan","Blouse","Jacket","Jeans","Maxi","Floral","Denim","Sweatshorts","Polka","Shawl","Bodycon"]
CLASS_LIST = CLASS_LIST_21
# IMG_PATH = FOLDER_PATH + FILEPATH
# print(IMG_PATH)
# img, height, width = readImg(IMG_PATH)
# g_kernel = gaussian_kernel(height, width,kernel_relative_size=0.01)


In [5]:
def gaussian_kernel(img_height, img_width, kernel_relative_size = 0.1, verbose=False):
    xs = scipy.signal.gaussian(img_width, std= img_width * kernel_relative_size)
    ys = scipy.signal.gaussian(img_height, std=img_height * kernel_relative_size)
    kernel = np.array([[x*y for x in xs] for y in ys])
    return kernel

In [6]:
def readImg(filepath):
    img = cv2.imread(filepath)
    # img.resize()
    height = len(img)
    width= len(img[0])
    return img, height, width

In [7]:
from collections import defaultdict
import pandas as pd
import numpy as np
import os
datasetdir = 'C:\\Users\\mdc20\\Downloads\\img\\'
def constructImageClassDataFrame(class_list = CLASS_LIST):
    # Args: 
    #   Shape: Image shape (2D)
    #   min_class_occurence: number of times class must occur in labeled dir in order to add it to the final sclass list
    os.chdir(datasetdir)
    
    sub_dirs = [d for d in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(datasetdir, d))]
    
    arr = [[labeled_dir, file] for labeled_dir in sub_dirs for file in os.listdir(labeled_dir) ]
    print("Done parsing through directories")
    df = pd.DataFrame(data=arr, columns=["folder","filename"])

    df['filepath'] = datasetdir + df['folder'] + '\\' + df['filename']
    df['labels'] = df['folder'].apply(lambda x : x.split('_') if len(x.split('_')) > 0 else None)

    print(np.shape(df))
    df = df[df.labels.notnull()]


    return df

In [8]:
df = constructImageClassDataFrame()

Done parsing through directories
(289223, 4)


In [9]:
df.head()

,folder,filename,filepath,labels
0,1981_Graphic_Ringer_Tee,img_00000001.jpg,C:\Users\mdc20\Downloads\img\1981_Graphic_Ring...,"[1981, Graphic, Ringer, Tee]"
1,1981_Graphic_Ringer_Tee,img_00000002.jpg,C:\Users\mdc20\Downloads\img\1981_Graphic_Ring...,"[1981, Graphic, Ringer, Tee]"
2,1981_Graphic_Ringer_Tee,img_00000003.jpg,C:\Users\mdc20\Downloads\img\1981_Graphic_Ring...,"[1981, Graphic, Ringer, Tee]"
3,1981_Graphic_Ringer_Tee,img_00000004.jpg,C:\Users\mdc20\Downloads\img\1981_Graphic_Ring...,"[1981, Graphic, Ringer, Tee]"
4,1981_Graphic_Ringer_Tee,img_00000005.jpg,C:\Users\mdc20\Downloads\img\1981_Graphic_Ring...,"[1981, Graphic, Ringer, Tee]"


In [10]:
from numpy.random import choice
a = [1, 2, 3, 4]
b = [0.1, 0.5, 0.7, 0.9]
b = b / np.sum(b)

draw = choice(a, 3, replace=True, p=b)
print(draw)

[4 4 4]


In [11]:
from numpy.random import choice
import time
# from sklearn.neighbors import NearestNeighbors

from sklearn.cluster import KMeans
def readImg(filepath, resize_shape = (224,224)):
    img = cv2.imread(filepath)
    if img is not None:
        r_img = cv2.resize(img, resize_shape)
        return r_img
    else:
        return img

def get_flatted_normalized_gaussian_kernel(kernel: list[list[float]]):
    kernel = kernel.reshape(-1)
    kernel = kernel / np.sum(kernel)
    return kernel 
    

def get_draw(img_colors, normed_kernel = None, sample_size = 1000):
    if normed_kernel is not None:
        draw_idx = choice(a=224**2, size=sample_size, replace=True, p=normed_kernel)
    else:
        draw_idx = choice(a=224**2, size=sample_size, replace=True)
    # Get rows specified by draw idx for img_colors
    return img_colors[draw_idx]


def rgb_to_hex(r, g, b):
    return ('{:02X}' * 3).format(r, g, b)


def get_knn_for_colors_list(colors):
    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans.fit(colors)
    return kmeans.cluster_centers_

def get_dominant_colors(filepath, normed_kernel = None):    
    img = readImg(filepath=filepath, resize_shape=(224,224))
    if img is None:
        return list()
    if normed_kernel is None:
        kernel = gaussian_kernel(224, 224, kernel_relative_size = 0.1, verbose=False)
        normed_kernel = get_flatted_normalized_gaussian_kernel(kernel)
    # Resize the image input to be a list of lenght width*height
    img_colors = np.array(img.reshape((-1, 3)), dtype=float)

    draw = get_draw(img_colors=img_colors, normed_kernel=normed_kernel, sample_size=200)
    # return draw
    knn_colors = get_knn_for_colors_list(draw)
    # knn_colors = []
    hex_vals = list()
    for color in knn_colors:
        # These colors are encoded with BGR
        blue = int (color[0])
        green = int (color[1])
        red = int (color[2])
        hex_val = rgb_to_hex(red, green, blue)
        hex_vals.append(hex_val)
    return hex_vals
    # g_kernel = gaussian_kernel(height, width, kernel_relative_size=0.05)


# df['hex_code'] = df['filepath'].apply(lambda x: get_dominant_color(x))


kernel = gaussian_kernel(224, 224, kernel_relative_size = 0.1, verbose=False)
normed_kernel = get_flatted_normalized_gaussian_kernel(kernel)
# hex_vals = get_dominant_colors("E:\\img_highres\\Abstract_Print_Mesh_Tee\\img_00000003.jpg", normed_kernel=normed_kernel)

df_test_batch = df.sample(100)
before = time.time()
df_test_batch['hex_code'] = df_test_batch['filepath'].apply(lambda x: get_dominant_colors(x, normed_kernel=normed_kernel))
after = time.time()
print (after - before)

3.1455142498016357


In [12]:
# Now we do the whole dataframe
mongo_prod_batch_df = df
kernel = gaussian_kernel(224, 224, kernel_relative_size = 0.1, verbose=False)
normed_kernel = get_flatted_normalized_gaussian_kernel(kernel)
before = time.time()
mongo_prod_batch_df['hex_codes'] = mongo_prod_batch_df['filepath'].apply(lambda x: get_dominant_colors(filepath=x, normed_kernel=normed_kernel))
after = time.time()
print (after - before)
mongo_prod_batch_df.to_csv('C:\\Users\\mdc20\\full_labels.csv')

C:\Users\mdc20\AppData\Local\Temp\ipykernel_29352\2493317562.py:36: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans.fit(colors)


In [ ]:
mongo_prod_batch_df.to_csv('C:\\Users\\mdc20\\labeled_filenames_2.csv')


In [78]:
kernel = gaussian_kernel(224, 224, kernel_relative_size = 0.1, verbose=False)
normed_kernel = get_flatted_normalized_gaussian_kernel(kernel)
before = time.time()
df['hex_codes'] = df['filepath'].apply(lambda x: get_dominant_colors(filepath=x, normed_kernel=normed_kernel))
after = time.time()
print (after - before)
df.to_csv('C:\\Users\\mdc20\\labeled_filenames.csv')


KeyboardInterrupt: 

In [16]:
new_df.head(10)

244300    B29C90
114934    978C8B
19860     875D5C
36349     4E4F5A
105348    CDACA2
13091     403F3F
34121     837F82
232415    D8CCC2
166516    BFB6AA
75790     BFBFBF
Name: filepath, dtype: object

In [61]:
filter_indices = [1, 2]
array = np.array([[1, 2, 3, 4, 5], 
                  [10, 20, 30, 40, 50], 
                  [100, 200, 300, 400, 500]])
array[filter_indices]

array([[ 10,  20,  30,  40,  50],
       [100, 200, 300, 400, 500]])